In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [12]:
import pyrealsense2 as rs
import cv2
import numpy as np
import sys
import os
import multiprocessing as mp
import pandas as pd
import tqdm 
from cv2 import VideoWriter, VideoWriter_fourcc

In [10]:
# start the process of extracting the video information for each video

def get_color_from_bag(BAG_File):

    pipeline = rs.pipeline()
    config = rs.config()

    rs.config.enable_device_from_file(config, BAG_File, repeat_playback=False)

    config.enable_all_streams()
    profile = pipeline.start(config)

    # create alignment object
    align_to = rs.stream.color
    align = rs.align(align_to)

    # inform the device that this is not live streaming from camera
    playback = profile.get_device().as_playback()
    playback.set_real_time(False)
    duration = playback.get_duration()

    true_frame_number = []
    frame_number = []
    time_st = []

    num_frame = 0


    Color_Frames = []#{}

    try:
        while True:
            frames = pipeline.wait_for_frames(100)  #get frame from file 

            this_frame = frames.get_frame_number()  #get frame number 

            if (num_frame != 0) and (true_frame_number[-1] == this_frame): #verify that frame number is not repeated 
                #if frame number is repeated then replace the stored information 
                aligned_frames = align.process(frames)

                #take color and depth from frame, if any to these is not available then skip the frame
                aligned_depth = aligned_frames.get_depth_frame()
                aligned_color = aligned_frames.get_color_frame()

                # validate that both frames are available
                if not aligned_depth or not aligned_color:
                    continue

                time_stamp = frames.get_timestamp()
                true_frame_number[-1] = frames.get_frame_number()
                time_st[-1] = time_stamp 

                # transform to np array

                color_data = np.asanyarray(aligned_color.as_frame().get_data(), dtype=np.int)
                #depth_data = np.asanyarray(aligned_depth.as_frame().get_data(), dtype=np.int)
                # adjust depth data in meters
                #depth_data *= depth_scale

                Color_Frames[-1] = color_data

            else:
                #if frame number is not repeated then append the stored information 
                aligned_frames = align.process(frames)

                #take color and depth from frame, if any to these is not available then skip the frame
                aligned_depth = aligned_frames.get_depth_frame()
                aligned_color = aligned_frames.get_color_frame()

                # validate that both frames are available
                if not aligned_depth or not aligned_color:
                    continue

                time_stamp = frames.get_timestamp()
                true_frame_number.append(frames.get_frame_number())
                time_st.append(time_stamp )

                # transform to np array

                color_data = np.asanyarray(aligned_color.as_frame().get_data(), dtype=np.int)
                #depth_data = np.asanyarray(aligned_depth.as_frame().get_data(), dtype=np.int)
                # adjust depth data in meters
                #depth_data *= depth_scale

                Color_Frames.append(color_data)
                #Depth_Frames.append(depth_data

                frame_number.append(num_frame)
                num_frame += 1

    except RuntimeError:
        pass
    finally:
        pipeline.stop()

    duration_movie = duration.total_seconds()
    FPS = num_frame/duration_movie
    height, width,_ =  Color_Frames[0].shape

    color_file = BAG_File[:-4]+'_color.mp4'

    video = VideoWriter(color_file, -1, int(FPS), (width,height))

    for k in range(num_frame):
        frame_to_save = Color_Frames[k].astype('uint8')
        video.write(frame_to_save)

    video.release()    


    cvs_frame_info = BAG_File[:-4]+'_frameInfoColor.csv'
    df_cols = ['Actual_Frame_Number', 'Frame_Time_Stamp', 'Frame_Number_in_Video']
    df = pd.DataFrame(columns=df_cols)
    df['Actual_Frame_Number'] = true_frame_number
    df['Frame_Time_Stamp'] = (np.array(time_st)-time_st[0])/1000
    df['Frame_Number_in_Video'] = frame_number

    df.to_csv(cvs_frame_info)


In [15]:
path = r'C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\Healthy'
Files = os.listdir(path)            
ext=('.bag')
Files = [i for i in Files if i.endswith(tuple(ext))]
for k,f in enumerate(Files):
    Files[k] = os.path.join(path,f)

pbar = tqdm.tqdm(total=len(Files))
    
for f in Files:
    get_color_from_bag(f)
    pbar.update(1)
    
# pool = mp.Pool(mp.cpu_count())

# #tqdm.tqdm(pool.imap_unordered(get_color_from_bag, [BAG_File for BAG_File in Files]), total=len(Files))
# for _ in tqdm.tqdm(pool.imap_unordered(get_color_from_bag, [BAG_File for BAG_File in Files]), total=len(Files)):
#     pass
# #pool.map(get_color_from_bag, [BAG_File for BAG_File in Files])
# #pool.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 64/64 [06:54<00:00,  4.07s/it]